### Inicializando e checando o ecossistema de databases

In [1]:
#mongodb
!mongo --quiet --eval 'rs.initiate();'
!mongo --quiet --eval 'rs.isMaster().ismaster'

#redis
!redis-server --daemonize yes
!redis-cli ping

#cassandra
!cqlsh --execute="SELECT host_id, bootstrapped FROM system.local;"
!cqlsh --file=./cassandra/initialize.cql

{
	"info2" : "no configuration specified. Using a default configuration for the set",
	"me" : "localhost:27017",
	"ok" : 1,
	"$clusterTime" : {
		"clusterTime" : Timestamp(1689792703, 1),
		"signature" : {
			"hash" : BinData(0,"AAAAAAAAAAAAAAAAAAAAAAAAAAA="),
			"keyId" : NumberLong(0)
		}
	},
	"operationTime" : Timestamp(1689792703, 1)
}
true
434:C 19 Jul 2023 18:51:55.129 # oO0OoO0OoO0Oo Redis is starting oO0OoO0OoO0Oo
434:C 19 Jul 2023 18:51:55.129 # Redis version=6.0.5, bits=64, commit=e50880bc, modified=1, pid=434, just started
434:C 19 Jul 2023 18:51:55.129 # Configuration loaded
PONG

 host_id                              | bootstrapped
--------------------------------------+--------------
 0aa65e45-8a3c-45b7-89c8-5465320f89a1 |    COMPLETED

(1 rows)


### Inicializando modulos, libs e variaveis

In [2]:
from pymongo import MongoClient
from pprintpp import pprint
import warnings
import json
from cassandra.cluster import Cluster
from libs.dispute_faker import generate_dispute

# Decoder personalizado para Cassandra
def dict_cleaner(items):
    result = {}
    for key, value in items:
        if 'gmt' in key:
            try:
                value = str(value*1000)
            except TypeError:
                value = None
        result[key] = value
    return result

warnings.filterwarnings('ignore')

mongoclient = MongoClient('localhost', 27017)
dispute_db = mongoclient["disputes"]
print(dispute_db)

cluster = Cluster()
cassandra = cluster.connect('disputes')
pprint(cassandra)


Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'disputes')


In [3]:
inserted = dispute_db['disputes'].insert_many([generate_dispute() for _ in range(2000)])
pprint(len(inserted.inserted_ids))

2000


In [13]:
query = dispute_db['disputes'].find({})
for dispute in query:
    dispute['id'] = str(dispute['_id'])
    dispute['arbitratorId'] = str(dispute['resolution']['arbitratorId']) if not None else ''
    dispute['decision'] = str(dispute['resolution']['decision']) if not None else ''
    dispute['status'] = str(dispute['resolution']['status'])
    dispute['itemId'] = str(dispute['item']['itemId'])
    dispute['skuId'] = str(dispute['item']['skuId'])
    dispute['id'] = str(dispute['_id'])
    dispute.pop('_id')
    dispute = json.dumps(dispute)
    cleaned = json.dumps(json.loads(dispute, object_pairs_hook=dict_cleaner))
    cassandra.execute(f"INSERT INTO disputes JSON'{cleaned}'").all()
pprint(cassandra.execute(f"SELECT COUNT(*) from disputes").all())

[Row(count=2000)]


In [14]:
!cqlsh --execute="SELECT * FROM disputes.disputes WHERE id='64b8313adf0a6c558a4bed13';"


 id                       | arbitratorid | buyerid | decision              | disputereason    | gmtcreated                      | gmtmodified                     | item                                                                                                                                                                                                                                            | itemid | resolution                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              